In [25]:
import math
import numpy as np
import torch
from torch import Tensor, nn, optim

In [38]:
def polynomial_gen(polynomial_coefficients: list, size, noise_std=0.1) -> tuple[Tensor, Tensor]:
    """
    生成 n 阶多项式数据
    .. math:: y = \\sum_{i=0}^{n} \\frac{a_i}{i!} x^i + \\epsilon
    :param polynomial_coefficients: a_i
    :param size: 生成的数据集大小
    :param noise_std: 多项式的噪声项
    :return: 特征张量与标签张量的元组
    """
    polynomial_coefficients = np.array(polynomial_coefficients)
    orders = np.arange(len(polynomial_coefficients))

    x = np.random.normal(size=(size, len(polynomial_coefficients)))
    features = np.power(x, orders) / np.vectorize(math.factorial)(orders)
    labels = features @ polynomial_coefficients
    labels += np.random.normal(scale=noise_std, size=labels.shape)
    return torch.from_numpy(features).to(torch.float32), torch.from_numpy(labels).to(torch.float32)

In [39]:
coeffs = [5.0, 1.2, -3.4, 5.6]
SIZE = 1000
EPOCHS = 1000

features, labels = polynomial_gen(coeffs, SIZE)

In [40]:
net = nn.Sequential(nn.Linear(in_features=len(coeffs), out_features=1, bias=False))
net.apply(lambda m: nn.init.normal_(m.weight, mean=0, std=0.01) if isinstance(m, nn.Linear) else None)

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [65]:
net[0].weight.data.numpy().squeeze().tolist()

[-0.0015493236714974046,
 -0.016542261466383934,
 0.010300985537469387,
 0.0016125075053423643]